In [1]:
import io, os, sys, types

In [2]:
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [4]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [5]:
# 1.10 Running scripts

import humansize

print(humansize.__name__)
print(__name__)
if(__name__ == '__main__'):
    print("Running as a standalone program")
    print(humansize.approximate_size(4096, True))

importing Jupyter notebook from humansize.ipynb
humansize
__main__
Running as a standalone program
4.0 KiB


In [6]:
# 1.7.1 Catching import errors

try:
    import chardet
except ImportError:
    chardet = None
    
if chardet:
    print ("chardet is imported")
else:
    print ("chardet is not imported")
    
try:
    from lxml import etree
    print("lxml is imported")
except ImportError:
    import xml.etree.ElementTree as etree
    print("xml.etree.ElementTree is imported")

chardet is imported
lxml is imported


In [12]:
# 2.2 Booleans - Booleans are either True and False.

# True is 1; False is 0.
print("True + True = ", True + True)
print("False + False = ", False + False)

# 2.3 Numbers
print(type(1))
print(type(2.0))
print(isinstance(1, int))
print(isinstance(1 + 2.0, float))

# 2.3.1 Coercing integers to floats and vice-versa
print(type(float(2)))
print(int(2.5))
print(int(-2.5))

# 2.3.2 Common numerical operations
print(11/2)
print(11//2)

print(-11//2) # rounding down from -5.5 to -6
print(11.0/2)
print(11.0//2)
print(11 % 2)
print(11**2)

True + True =  2
False + False =  0
<class 'int'>
<class 'float'>
True
True
<class 'float'>
2
-2
5.5
5
-6
5.5
5.0
1
121


In [7]:
# 2.3.3 Fractions

import fractions

x = fractions.Fraction(1, 3)
print(x)
print(type(x))
print(x * 2)
print(x * 3)
print(type(x * 3))
print(fractions.Fraction(6, 4))

1/3
<class 'fractions.Fraction'>
2/3
1
<class 'fractions.Fraction'>
3/2


In [8]:
# 2.3.4 Trigonometry

import math
print(math.pi)
print(type(math.pi))
print(math.sin(math.pi / 2))
print(math.tan(math.pi / 4))

3.141592653589793
<class 'float'>
1.0
0.9999999999999999


In [9]:
# 2.3.5 Numbers is a boolean context

import fractions

def is_it_true(anything):
    if anything:
        print("yes, it's true")
    else:
        print("no, it's false")
        
print(is_it_true(1))
print(is_it_true(0))
print(is_it_true(-1))
print(is_it_true(0.0))
print(is_it_true(fractions.Fraction(1, 2)))
print(is_it_true(fractions.Fraction(0, 2)))

yes, it's true
None
no, it's false
None
yes, it's true
None
no, it's false
None
yes, it's true
None
no, it's false
None


In [20]:
# 2.4 Lists

# 2.4.1 Creating a list

a_list = ['a', 'b', 'new', 'mpilgrim', 'z', 'example', 'new']
print(a_list)
print(a_list[0])
print(a_list[4])
# print(a_list[5]) # IndexError
print(a_list[-1])
print(a_list[-3])
# print(a_list[-6]) # IndexError
print(len(a_list))

# 2.4.2 Slicing a list
print(a_list[1:3])
print(a_list[1:-1])
print(a_list[0:3])
print(a_list[:3])
print(a_list[3:])
print(a_list[:])

# 2.4.3 Adding items to a list
a_list = ['a']
a_list = a_list + [2.0, 3]
print(a_list)
a_list.append(True)
print(a_list)
a_list.extend(['four', 'Q'])
print(a_list)
a_list.insert(0, 'Q')
print(a_list)
a_list.insert(3, 'P')
print(a_list)
a_list.append(['g', 'h', 'i'])
print(a_list)
print(a_list[-1])

# 2.4.4 Searching for values in a list

a_list = ['a', 'b', 'new', 'mpilgrim', 'new']
print(a_list.count('new'))
print('new' in a_list)
print('c' in a_list)
print(a_list.index('mpilgrim'))
# print(a_list.index('c')) # raise ValueError: 'c' is not in list

# 2.4.5 Removing items from a list
print(a_list[1])
del a_list[1]
print(a_list)
print(a_list[1])

a_list = ['a', 'b', 'new', 'mpilgrim', 'new']
a_list.remove('new')
print(a_list)
a_list.remove('new')
print(a_list)

# a_list.remove('new') # ValueError: list.remove(x): x not in list

# 2.4.6 Removing items from a list using pop
a_list = ['a', 'b', 'new', 'mpilgrim']
print(a_list.pop())
print(a_list)
a_list.pop(1)
print(a_list)
a_list.pop()
print(a_list)
a_list.pop()
print(a_list)
# a_list.pop() # IndexError: pop from empty list

# 2.4.7 Lists in a boolean context
print(is_it_true([]))
print(is_it_true(['a']))
print(is_it_true([False]))


['a', 'b', 'new', 'mpilgrim', 'z', 'example', 'new']
a
z
new
z
7
['b', 'new']
['b', 'new', 'mpilgrim', 'z', 'example']
['a', 'b', 'new']
['a', 'b', 'new']
['mpilgrim', 'z', 'example', 'new']
['a', 'b', 'new', 'mpilgrim', 'z', 'example', 'new']
['a', 2.0, 3]
['a', 2.0, 3, True]
['a', 2.0, 3, True, 'four', 'Q']
['Q', 'a', 2.0, 3, True, 'four', 'Q']
['Q', 'a', 2.0, 'P', 3, True, 'four', 'Q']
['Q', 'a', 2.0, 'P', 3, True, 'four', 'Q', ['g', 'h', 'i']]
['g', 'h', 'i']
2
True
False
3
b
['a', 'new', 'mpilgrim', 'new']
new
['a', 'b', 'mpilgrim', 'new']
['a', 'b', 'mpilgrim']
mpilgrim
['a', 'b', 'new']
['a', 'new']
['a']
[]
no, it's false
None
yes, it's true
None
yes, it's true
None


In [135]:
# 2.5 Tuples

a_tuple = ("a", "b", "mpilgrim", "z", "example")
print(a_tuple)
print(a_tuple[0])
print(a_tuple[-1])
print(a_tuple[1:3])

# a_tuple.append("new") #AttributeError: 'tuple' object has no attribute 'append'
# a_tuple.remove("z") # AttributeError: 'tuple' object has no attribute 'remove'
print(a_tuple.index("example"))
print("z" in a_tuple)

# 2.5.1 Tuples in a boolean context
print(is_it_true(()))
print(is_it_true(("a", "b")))
print(is_it_true((False,)))
print(type((False)))
print(type((False,)))

# 2.5.2 Assigning multiple values at once
v = ('a', 2, True)
(x, y, z) = v
print(x)
print(y)
print(z)

(MONDAY, TUESDAY, WEDNESDAY, THURSDAY, FRIDAY, SATURDAY, SUNDAY) = range(7) 
print(MONDAY)
print(SATURDAY)
print(SUNDAY)

('a', 'b', 'mpilgrim', 'z', 'example')
a
example
('b', 'mpilgrim')
4
True
no, it's false
None
yes, it's true
None
yes, it's true
None
<class 'bool'>
<class 'tuple'>
a
2
True
0
5
6


In [22]:
# 2.6 Sets

# 2.6.1 Creating a set
a_set = {1}
print(a_set)
print(type(a_set))
a_set = {1, 2}
print(a_set)

# Create a set out of a list
a_list = ['a', 'b', 'mpilgrim', True, False, 42]
a_set = set(a_list)
print(a_set)
a_set = set()      # It is an empty set
print(len(a_set))
print(type(a_set))
print(type({}))   # It is an empty dict type

# 2.6.2 Modifying a set
a_set = {1, 2}
a_set.add(4)
a_set.add(1)
print(a_set)

# update()
a_set = {1, 2, 3}
print(a_set)
a_set.update({2, 4, 6})
print(a_set)
a_set.update({3, 6, 9}, {1, 2, 3, 5, 8, 13})
print(a_set)
a_set.update({10, 20, 30})
print(a_set)

# 2.6.3 Removing items from a set
a_set = {1, 3, 6, 10, 15, 21, 28, 36, 45}
a_set.discard(10)
print(a_set)
a_set.discard(10)
print(a_set)
a_set.remove(21)
print(a_set)
# a_set.remove(21)  # KeyError: 21
# If you call the discard() with a value that doesn't exist in the set, it does nothing.
# The remove() method raises a KeyError wif the value does not exist.

# poo() method, there is no order, it is ocmpletely arbitrary.
a_set = {1, 3, 6, 10, 15, 21, 28, 36, 45}
print(a_set.pop())
print(a_set.pop())
print(a_set.pop())
print(a_set)
a_set.clear()
print(a_set)
# a_set.pop() # KeyError: 'pop from an empty set'

# 2.6.4
a_set = {2, 4, 5, 9, 12, 21, 30, 51, 76, 127, 195}
print(30 in a_set)
print(31 in a_set)

b_set = {1, 2, 3, 5, 6, 8, 9, 12, 15, 17, 18, 21}
print(a_set.union(b_set))
print(a_set.intersection(b_set))
print(a_set.difference(b_set))
print(a_set.symmetric_difference(b_set))
print(b_set.symmetric_difference(a_set))
print(b_set.symmetric_difference(a_set) == a_set.symmetric_difference(b_set))
print(b_set.union(a_set) == a_set.union(b_set))
print(b_set.intersection(a_set) == a_set.intersection(b_set))
print(b_set.difference(a_set) == a_set.difference(b_set))

a_set = {1, 2, 3}
b_set = {1, 2, 3, 4}
print(a_set.issubset(b_set))
print(b_set.issuperset(a_set))
a_set.add(5)
print(a_set.issubset(b_set))
print(b_set.issuperset(a_set))

# 2.6.5 Sets in a boolean context

def is_it_true(anything):
    if anything:
        print("yes, it's true")
    else:
        print("no, it's false")

print(is_it_true(set()))
print(is_it_true({'a'}))
print(is_it_true({False}))


{1}
<class 'set'>
{1, 2}
{False, True, 42, 'b', 'a', 'mpilgrim'}
0
<class 'set'>
<class 'dict'>
{1, 2, 4}
{1, 2, 3}
{1, 2, 3, 4, 6}
{1, 2, 3, 4, 5, 6, 8, 9, 13}
{1, 2, 3, 4, 5, 6, 8, 9, 10, 13, 20, 30}
{1, 3, 36, 6, 45, 15, 21, 28}
{1, 3, 36, 6, 45, 15, 21, 28}
{1, 3, 36, 6, 45, 15, 28}
1
3
36
{6, 10, 45, 15, 21, 28}
set()
True
False
{1, 2, 195, 4, 5, 3, 6, 8, 9, 12, 76, 15, 17, 18, 21, 30, 51, 127}
{2, 5, 9, 12, 21}
{195, 4, 76, 51, 30, 127}
{1, 3, 195, 6, 4, 8, 76, 15, 17, 18, 51, 30, 127}
{1, 195, 4, 3, 6, 8, 76, 15, 17, 18, 51, 30, 127}
True
True
True
False
True
True
False
False
no, it's false
None
yes, it's true
None
yes, it's true
None


In [32]:
# 2.7 Dictionaries

# 2.7.1 Creating a dictionary
a_dict = {'server' : 'db.diveintopython3.org', 'database' : 'mysql'}
print(a_dict)
print(a_dict['server'])
print(a_dict['database'])
# a_dict['db.diveintopython3.org'] # KeyError: 'db.diveintopython3.org'

# 2.7.2 Modifying a dictionary
a_dict['database'] = 'blog'
print(a_dict)
a_dict['user'] = 'mark'
print(a_dict)
a_dict['user'] = 'dora'
print(a_dict)
a_dict['User'] = 'mark'
print(a_dict)

# 2.7.3 Mixed-Value dictionaries
SUFFIXES = {1000: ['KB', 'MB', 'GB', 'TB', 'PB', 'EB', 'ZB', 'YB'],
            1024: ['KiB', 'MiB', 'GiB', 'TiB', 'PiB', 'EiB', 'ZiB', 'YiB']}
print(len(SUFFIXES))
print(1000 in SUFFIXES)
print(SUFFIXES[1000])
print(SUFFIXES[1000][3])
print(SUFFIXES[1024])

# 2.7.4 Dictionaries in a boolean context
print(is_it_true({}))
print(is_it_true({'a' : 1}))



{'server': 'db.diveintopython3.org', 'database': 'mysql'}
db.diveintopython3.org
mysql
{'server': 'db.diveintopython3.org', 'database': 'blog'}
{'server': 'db.diveintopython3.org', 'database': 'blog', 'user': 'mark'}
{'server': 'db.diveintopython3.org', 'database': 'blog', 'user': 'dora'}
{'server': 'db.diveintopython3.org', 'database': 'blog', 'user': 'dora', 'User': 'mark'}
2
True
['KB', 'MB', 'GB', 'TB', 'PB', 'EB', 'ZB', 'YB']
TB
['KiB', 'MiB', 'GiB', 'TiB', 'PiB', 'EiB', 'ZiB', 'YiB']
no, it's false
None
yes, it's true
None


In [38]:
# 2.8 None
print(type(None))
print(None == False)
print(None == 0)
print(None == None)

# 2.8.1 None in a boolean context
# None is false, not None is true
print(is_it_true(None))
print(is_it_true(not None))

<class 'NoneType'>
False
False
True
no, it's false
None
yes, it's true
None
